# Enigma Machine Design



## Inside the Enigma

The enigma machine is a fairly complex encryption machine which consists of four main sections:

![major-components](../images/mission-x-challenge/intro-inside-enigma.png)



# Enigma Machine Code

## The Keyboard

The keyboard is used to retrieve the user input. The Enigma machine is a symmetric encryption machine. Which means that it can be used to both encrypt or decrypt a message using the same settings. The keyboard is hence used to either enter the plaintext that needs to be encrypted or the ciphertext that needs to be decrypted.

They keyboard consists of 26 keys for each letter of the alphabet. This means that encrypted messages will be joined up without any spaces or punctuation signs.

Notice how the keyboard starts with the letters `QWERTZ` instead of `QWERTY`. This is due to the fact that in the German language, the letter Z is more often used than the letter.

In [ ]:
import string
import random

class Letter:
    """Letter class : contains the string representation of a letter and its position in the alphabet"""
    def __init__(self, y) -> None:
        if isinstance(y, str):
            self.letter = y.lower()
            self.index = Letter.a2i(self.letter)
        elif isinstance(y, int):
            self.index = y
            self.letter = Letter.i2a(y)
        else:
            raise ValueError("Not Implemented")

    def __add__(self, y):
        q = self.get_type(y)
        x = (26 + self.index + q) % 26
        return Letter(string.ascii_lowercase[x])

    def __sub__(self, y):
        q = self.get_type(y)
        x = (26 + self.index - q) % 26
        return Letter(string.ascii_lowercase[x])

    def __eq__(self, y):
        return self.letter == y.letter

    def __str__(self):
        return self.letter

    def get_type(self, y):
        if isinstance(y, Letter):
            q = y.index
        else:
            q = y
        return q

    @staticmethod
    def random():
        return Letter(random.choice(string.ascii_lowercase))

    @staticmethod
    def a2i(a: str) -> int:
        return string.ascii_lowercase.find(a)

    @staticmethod
    def i2a(i: int) -> str:
        return string.ascii_lowercase[i]


class Pair:
    """A Plugboard pair"""
    def __init__(self, A: str, B: str) -> None:
        self.a = Letter(A)
        self.b = Letter(B)

    def get(self, x: Letter) -> Letter:
        if self.a == x:
            return self.b
        elif self.b == x:
            return self.a
        else:
            return None

    def __str__(self):
        return f"{self.a}{self.b}"

In [ ]:

def test_letter_from_str():
    tests = []
    for i in range(26):
        character = string.ascii_lowercase[i]
        index = i
        tests.append({"letter": character, "index": index})

    for test in tests:
        letter = Letter(test["letter"])
        assert letter.letter == test["letter"] and letter.index == test["index"]


def test_letter_from_index():
    tests = []
    for i in range(26):
        character = string.ascii_lowercase[i]
        index = i
        tests.append({"letter": character, "index": index})

    for test in tests:
        letter = Letter(test["index"])
        assert letter.letter == test["letter"] and letter.index == test["index"]


def test_pairs():
    tests = [
        {"pair": {"A": "x", "B": "c"}, "off": "t"},
        {"pair": {"A": "r", "B": "t"}, "off": "g"},
        {"pair": {"A": "r", "B": "t"}, "off": "l"},
    ]

    for test in tests:
        p = test["pair"]
        pair = Pair(p["A"], p["B"])

        a = Letter(p["A"])
        b = Letter(p["B"])
        off = Letter(test["off"])

        assert(pair.get(a) == b and pair.get(b) == a and pair.get(off) == None)


test_letter_from_str()
test_letter_from_index()
test_pairs()



## The plugboard

Once a key is pressed on the keyboard, it goes through the plugboard which provides the first stage of the encryption process. It is based on the principles of a substitution cipher, a form of transposition encryption.

To setup the keyboards, short wires are used to connect pairs of letters that will be permuted. For instance on the picture below the letter W will be replaced with a D and the letter D with a W as a (red) wire is used to connect these two letters/plugs. Similarly, letter V will become letter Z and Z will become V.

In a code book the plugboard settings would be recorded as follows: DW VZ

![plugboard](../images/mission-x-challenge/intro-plugboard.jpeg)


In [ ]:
class Plugboard:
    def __init__(self, pairs: str) -> None:
        pairs = pairs.split(" ")
        self.pairs = [Pair(*p) for p in pairs if p]

    def encode(self, x: Letter):
        for pair in self.pairs:
            ret = pair.get(x)
            if ret:
                return ret
        return x

    def active_len(self):
        return len(self.pairs)

    def randomize(self, pairs=10):
        alphabet = string.ascii_lowercase
        while self.active_len() < pairs:
            a = random.choice(alphabet)
            alphabet = alphabet.replace(a, "")
            b = random.choice(alphabet)
            alphabet = alphabet.replace(b, "")
            self.pairs.append(Pair(a, b))

    def __str__(self):
        return " ".join(map(lambda p: str(p), self.pairs))

In [ ]:
def test_plugboard_init():
    tests = [
        {
            "sequence": "AB CD EF",
            "len": 3
        }
    ]

    for test in tests:
        p = Plugboard(test["sequence"])

        assert(p.active_len() == test["len"])

def test_plugboard_init_random():
    p = Plugboard("")
    p.randomize()

    assert(p.active_len() == 10)

test_plugboard_init()
test_plugboard_init_random()




## The Rotors

After the plugboard, the letter goes through the three rotors in order (from right to left), each of them changing it differently using a combination of transposition cipher and Caesar cipher! On the Enigma M3 there are three rotor slots and five rotors to choose from. Each rotor is identified using a Roman numeral from I to V. This provides a few settings of the Enigma machine: which rotors to use, and in which order to position them. In a code book this setting would be recorded as IV II III (Left, Middle and Right rotors).

Each of the five rotors encrypt the letter differently using a transposition/permutation cipher and can be connected in the Enigma machine with a different Ring setting. Another setting is the initial position of the rotors: Which letters are you going to set each rotor to begin with (e.g. A/B/C../Z sometimes recorded in a codebook using numbers (01 for A, 02 for B up to 26 for Z). This creates a Caesar Shift (Caesar Cipher). On an Enigma machine, you can change the position of the rotors by turning the three wheels.

Different versions of Enigma (e.g. M4) included four rotors which made the encryption process and the number of possible settings even bigger.

On our Enigma M3 emulator, you can click on the rotors to access the Enigma rotors settings:

![rotors](../images/mission-x-challenge/intro-rotors.jpeg)

![settings](../images/mission-x-challenge/intro-settings.jpeg)


In [ ]:
class Rotor:
    """Enigma rotor class"""

    def __init__(
        self,
        name: str,
        wiring: str,
        turnovers: str,
        reflector=False,
        position="A",
        ringstellung="A",
        rotate=True,
    ) -> None:
        self.name = name
        self.wiring = [Letter(wire) for wire in wiring]
        self.turnovers = [Letter(turnover) for turnover in turnovers]
        self.position = Letter(position)
        self.ringstellung = Letter(ringstellung)
        self.reflector = reflector
        self.can_rotate = rotate and not reflector

    def randomize(self):
        """Randomize the machine position"""
        self.position = Letter.random()
        self.ringstellung = Letter.random()

    def encode(self, letter: Letter):
        """From the plugboard to the reflector"""
        wiring_letter = letter + self.position - self.ringstellung
        offset = self.wiring[wiring_letter.index] - self.position + self.ringstellung
        return offset

    def decode(self, letter: Letter):
        """From the reflector to the plugboard"""
        wiring_letter = letter + self.position - self.ringstellung
        offset = Letter(self.wiring.index(wiring_letter)) - self.position + self.ringstellung
        return offset

    def rotate(self):
        """Rotate one time this rotor"""
        if not self.reflector and self.can_rotate:
            self.position += 1

    def __str__(self):
        return (f"{self.name}\n"
            f"Turnover: {''.join([str(t) for t in self.turnovers])}\n"
            f"Wiring: {''.join([str(w) for w in self.wiring])}\n"
            f"Position: {self.position}\n")


class Rotors:
    # M3 
    class M3:
        ETW     = Rotor("ETW", "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "", rotate=False)
        I       = Rotor("I", "EKMFLGDQVZNTOWYHXUSPAIBRCJ", "Q")
        II      = Rotor("II", "AJDKSIRUXBLHWTMCQGZNPYFVOE", "E")
        III     = Rotor("III", "BDFHJLCPRTXVZNYEIWGAKMUSQO", "V")
        IV      = Rotor("IV", "ESOVPZJAYQUIRHXLNFTGKDCMWB", "J")
        V       = Rotor("V", "VZBRGITYUPSDNHLXAWMJQOFECK", "Z")
        VI      = Rotor("VI", "JPGVOUMFYQBENHZRDKASXLICTW", "ZM")
        VII     = Rotor("VII", "NZJHGRCXMYSWBOUFAIVLPEKQDT", "ZM")
        VIII    = Rotor("VIII", "FKQHTLXOCBJSPDZRAMEWNIUYGV", "ZM")
        UKWB    = Rotor("UKWB", "YRUHQSLDPXNGOKMIEBFZCWVJAT", "", reflector=True)
        UKWC    = Rotor("UKWC", "FVPJIAOYEDRZXWGCTKUQSBNMHL", "", reflector=True)

    class M4(M3):
        Beta    = Rotor("Beta", "LEYJVCNIXWPBQMDRTAKZGFUHOS", "", rotate=False)
        Gamma   = Rotor("Gamma", "FSOKANUERHMBTIYCWLQPZXVGJD", "", rotate=False)
        UKWB    = Rotor("UKWB", "ENKQAUYWJICOPBLMDXZVFTHRGS", "", reflector=True)
        UKWC    = Rotor("UKWC", "RDOBJNTKVEHMLFCWZAXGYIPSUQ", "", reflector=True)



In [ ]:
def test_init_rotor():
    Rotor = Rotors.M3.I
    print(Rotor)


test_init_rotor()


## The Reflector

The reflector is another type of rotor inside the machine. Once the letter has gone through the three rotors from right to left, the reflector will reflect the electrical current back through the rotors, sending the encrypted letter through the rotors from left to right for another 3 stages of encryption and then through the plugboard again for a final substitution cipher. When going through the reflector, a permutation cipher is also applied to the letter.

Different versions of reflectors were used on different versions of Enigma machines. Each reflector would apply a different permutation cipher. Enigma M3 machines were equipped with either a UKW-B or UKW-C reflector. You can apply these two reflectors in the rotor settings window of our emulator (see screenshot above).

The diagram below shows the journey of a letter through the encryption process of an Enigma M3.

![reflector](../images/mission-x-challenge/intro-reflector.png)

In [ ]:
def test_init_reflector():
    Reflector = Rotors.M3.UKWB
    print(Reflector)

test_init_reflector()


## The lampboard
The lampboard is the final stage of the encryption process and is used to show the output (encrypted letter). It consists of 26 light bulbs, one for each letter of the alphabet.

## The Enigma Machine 

In [ ]:
class Machine:
    def __init__(
        self, name: str, rotors: list, plugboard: Plugboard, double_stepping=True
    ) -> None:
        self.name = name
        self.rotors = rotors
        self.plugboard = plugboard
        self.double_stepping = double_stepping

    def encode(self, sequence: str, split=5, debug=False) -> str:
        ret = ""
        for x in sequence:
            if x.lower() in string.ascii_lowercase:
                letter = Letter(x)
                j = [letter]

                # plugboard
                letter = self.plugboard.encode(letter)
                j.append(letter)

                self.rotate()

                # one way
                for rotor in self.rotors:
                    letter = rotor.encode(letter)
                    j.append(letter)

                # other way
                for rotor in reversed(self.rotors):
                    if not rotor.reflector:
                        letter = rotor.decode(letter)
                        j.append(letter)

                letter = self.plugboard.encode(letter)
                j.append(letter)

                ret += letter.letter

                # print(self)
                if debug:
                    print('->'.join(str(l) for l in j))

        if split > 0:
            ret = " ".join(ret[i : i + split] for i in range(0, len(ret), split))
        return ret

    def rotate(self):
        if self.double_stepping:
            self.rotate_double_stepping()
        else:
            self.rotate_normal()

    def rotate_double_stepping(self):
        rotors = list(filter(lambda r: r.can_rotate, self.rotors))
        assert len(rotors) == 3

        if self.rotors[2].position in self.rotors[2].turnovers:
            self.rotors[1].rotate()
            self.rotors[2].rotate()
            self.rotors[3].rotate()
        elif self.rotors[1].position in self.rotors[1].turnovers:
            self.rotors[1].rotate()
            self.rotors[2].rotate()
        else:
            self.rotors[1].rotate()

    def rotate_normal(self):
        for r, rotor in enumerate(self.rotors):
            if r == 0:
                rotor.rotate()

    def reset(self):
        for rotor in self.rotors:
            rotor.position = Letter("A")

    def set_position(self, position: list):
        position = list(reversed(position))
        assert len(position) == len(self.rotors)
        for rotor, p in zip(self.rotors, position):
            rotor.position = Letter(p)
    
    def get_position(self) -> list:
        positions = list(reversed(list(map(lambda r: r.position, filter(lambda r: r.can_rotate, self.rotors)))))
        return positions

    def set_ringstellung(self, ringstellung: list):
        ringstellung = list(reversed(ringstellung))
        assert len(ringstellung) == len(self.rotors)
        for rotor, p in zip(self.rotors, ringstellung):
            rotor.ringstellung = Letter(p)

    def get_ringstellung(self) -> list:
        return list(reversed(map(lambda r: r.ringstellung, self.rotors)))

    def set_rotor_position(self, name: str, position: str):
        for rotor in self.rotors:
            if rotor.name == name:
                rotor.position = Letter(position)

    def set_rotor_ringstellung(self, name: str, ringstellung: str):
        for rotor in self.rotors:
            if rotor.name == name:
                rotor.ringstellung = Letter(ringstellung)

    def __str__(self) -> str:
        reversed_rotor = list(reversed(self.rotors))
        return (f"Enigma {self.name}\n"
                f"Rotors: {'-'.join([r.name for r in reversed_rotor])}\n"
                f"Ringstellung: {'-'.join(str(r.ringstellung) for r in reversed_rotor)}\n"
                f"Positions: {'-'.join(str(r.position) for r in reversed_rotor)}\n"
                f"Plugboard: {str(self.plugboard)}")


In [ ]:
def test_hello_world():
    tests = [
        {"entry": "hello world", "result": "fsqsj fusta"},
        {"entry": "xgytk npnkq ssnxw kyf", "result": "alanm athis ontur ing"},
    ]

    for test in tests:
        p = Plugboard("qd fe rw jn il ps cm ax kg yu")

        M3 = Machine(
            "M3",
            [Rotors.M3.ETW, Rotors.M3.I, Rotors.M3.II, Rotors.M3.III, Rotors.M3.UKWC],
            p,
        )


        M3.set_rotor_position("I", "A")
        M3.set_rotor_position("II", "B")
        M3.set_rotor_position("III", "C")

        M3.set_rotor_ringstellung("I", "D")
        M3.set_rotor_ringstellung("II", "E")
        M3.set_rotor_ringstellung("III", "F")

        ciphertext = M3.encode(test["entry"])

        assert ciphertext == test["result"]

        M3.reset()

        M3.set_rotor_position("I", "A")
        M3.set_rotor_position("II", "B")
        M3.set_rotor_position("III", "C")

        M3.set_rotor_ringstellung("I", "D")
        M3.set_rotor_ringstellung("II", "E")
        M3.set_rotor_ringstellung("III", "F")

        plaintext = M3.encode(test["result"])

        assert plaintext == test["entry"]


def test_decode_M4_U534():
    # decode this : https://www.cryptomuseum.com/crypto/enigma/msg/p1030681.htm

    plugboard = Plugboard("AE BF CM DQ HU JN LX PR SZ VW")

    M4 = Machine(
        "M4",
        [
            Rotors.M4.ETW,
            Rotors.M4.VIII,
            Rotors.M4.VI,
            Rotors.M4.V,
            Rotors.M4.Beta,
            Rotors.M4.UKWC,
        ],
        plugboard,
    )

    M4.set_rotor_position("Beta", "C")
    M4.set_rotor_position("V", "D")
    M4.set_rotor_position("VI", "S")
    M4.set_rotor_position("VIII", "Z")

    M4.set_rotor_ringstellung("Beta", "E")
    M4.set_rotor_ringstellung("V", "P")
    M4.set_rotor_ringstellung("VI", "E")
    M4.set_rotor_ringstellung("VIII", "L")

    print(M4)

    ciphertext = """LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT
   GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN
   VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH
   VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO
   TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH
   DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML
   RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW"""

    plaintext = M4.encode(ciphertext, split=0, debug=False)

    assert (
        plaintext
        == """krkrallexxfolgendesistsofortbekanntzugebenxxichhabefolgelnbebefehlerhaltenxxjansterledesbisherigxnreichsmarschallsjgoeringjsetztderfuehrersieyhvrrgrzssadmiralyalsseinennachfolgereinxschriftlschevollmachtunterwegsxabsofortsollensiesaemtlichemassnahmenverfuegenydiesichausdergegenwaertigenlageergebenxgezxreichsleiteikktulpekkjbormannjxxobxdxmmmdurnhfkstxkomxadmxuuubooiexkp"""
    )


def test_double_stepping():
    plugboard = Plugboard("")

    M3 = Machine(
        "M3",
        [Rotors.M3.ETW, Rotors.M3.I, Rotors.M3.II, Rotors.M3.III, Rotors.M3.UKWC],
        plugboard,
    )

    M3.set_rotor_position("I", "Q")
    M3.set_rotor_position("II", "D")
    M3.set_rotor_position("III", "A")

    M3.encode("aaa")  # three rotation

    rotors = M3.get_position()

    assert (
        rotors[0].letter == "b" and rotors[1].letter == "f" and rotors[2].letter == "t"
    )

# http://wiki.franklinheath.co.uk/index.php/Enigma/Sample_Messages
def test_decode_M3_manual():
    M3 = Machine(
        "M3",
        [
            Rotors.M3.ETW,
            Rotors.M3.III,
            Rotors.M3.I,
            Rotors.M3.II,
            Rotor("UKWA", "EJMZALYXVBWFCRQUONTSPIKHGD", "", reflector=True),
        ],
        Plugboard("AM FI NV PS TU WZ"),
    )

    M3.set_rotor_position("II", "A")
    M3.set_rotor_position("I", "B")
    M3.set_rotor_position("III", "L")

    M3.set_rotor_ringstellung("II", string.ascii_lowercase[24 - 1])
    M3.set_rotor_ringstellung("I", string.ascii_lowercase[13 - 1])
    M3.set_rotor_ringstellung("III", string.ascii_lowercase[22 - 1])

    plaintext = M3.encode(
        "GCDSE AHUGW TQGRK VLFGX UCALX VYMIG MMNMF DXTGN VHVRM MEVOU YFZSL RHDRR XFJWC FHUHM UNZEF RDISI KBGPM YVXUZ",
        split=0,
    )

    assert (
        plaintext
        == "feindliqeinfanteriekolonnebeobaqtetxanfangsuedausgangbaerwaldexendedreikmostwaertsneustadt"
    )


def test_decode_enigma_K():
    K = Machine(
        "K",
        [
            Rotor("ETWK", "QWERTZUIOASDFGHJKPYXCVBNML", "", rotate=False),
            Rotor("KII", "CJGDPSHKTURAWZXFMYNQOBVLIE", "N"),
            Rotor("KI", "LPGSZMHAEOQKVXRFYBUTNICJDW", "Y"),
            Rotor("KII", "CJGDPSHKTURAWZXFMYNQOBVLIE", "N"),
            Rotor("UKWK", "IMETCGFRAYSQBZXWLHKDVUPOJN", "", reflector=True),
        ],
        Plugboard(""),
    )

    # 26 17 16 13
    K.set_rotor_ringstellung("KII", string.ascii_lowercase[13 - 1])
    K.set_rotor_ringstellung("KI", string.ascii_lowercase[16 - 1])
    K.set_rotor_ringstellung("KIII", string.ascii_lowercase[17 - 1])
    K.set_rotor_ringstellung("UKWK", string.ascii_lowercase[26 - 1])

    plaintext = K.encode("QSZVI DVMPN EXACM RWWXU IYOTY NGVVX DZ---", split=0)

    assert(
        plaintext == ""
    )

test_hello_world()
test_decode_M4_U534()
test_double_stepping()
test_decode_M3_manual()
# test_decode_enigma_K()

print("All tests passed")

## Enigma Machine M3

Code books were used by the Germans to list all the settings needed to set up the Enigma machines before starting to encrypt or decrypt messages. The Germans used to change the Enigma settings very regularly (e.g. once a day) so that if the Allies managed to break their code (find out the Enigma settings) they would only be able to use them for that day and would have to find the new settings every day. Code books were highly confidential documents as if a codebook was captured or reconstructed, messages could easily be decrypted.

An Enigma code book would have one page per month. The page would include all the settings for each day of the month with the first day of the month at the bottom of the page so that once used, a setting could be torn off the page.

The settings would indicate which rotors to use and in which order to connect them. Initially the Enigma machine came with a box of five rotors to choose from. On an Enigma M3, three out of the five rotors were connected. The M4 Enigma used four rotors chosen from a box of up to eight rotors.

The settings would also include the wheel settings (how to connect the rotors) and their initial position. Finally the settings would indicate which letters to connect by plugging cables on the plugboard.

**Enigma M3 Code Book (UKW-B Reflector) April 1940**

<p align="center">
<img src="../images/mission-x-challenge/intro-M3-codebook.jpeg" alt="Enigma M3 Code Book (UKW-B Reflector) April 1940" width="480"/>  
</p>

In [ ]:
M3 = Machine(
    "M3",
    [
        Rotors.M3.ETW,
        Rotors.M3.V,
        Rotors.M3.III,
        Rotors.M3.II,
        Rotors.M3.UKWB,
    ],
    plugboard = Plugboard("AO HI MU SN VX ZQ"),
)

def init_Enigma(M):
    M.reset()
    M.set_rotor_position("V", "F")
    M.set_rotor_position("III", "D")
    M.set_rotor_position("II", "V")

    M.set_rotor_ringstellung("V", "A")
    M.set_rotor_ringstellung("III", "K")
    M.set_rotor_ringstellung("II", "K")  
    print(M)
    return M

print('-'*79)
M3 = init_Enigma(M3)
plaintext = """HELLO WORLD"""
plaintext = plaintext.upper().replace(" ", "")
ciphertext = M3.encode(plaintext, split=0, debug=True)

print(f"{plaintext.upper()  =}")
print(f"{ciphertext.upper() =}")


In [ ]:
print('-'*79)
M3 = init_Enigma(M3)
message = M3.encode(ciphertext, split=0, debug=True)
print('-'*79)
print(f"{ciphertext.upper() =}")
print(f"{message.upper()    =}")


<a id="section_ID"></a>
## Table of Contents

1. [Enigma Machine](#1) 
2. [Test Enigma](#2)

In [ ]:
%load_ext autoreload
%autoreload 2

## Create a Enigma with Rotors and Reflector

In [ ]:
# create a function generate random order of 26 letters
import random
import numpy as np
import pandas as pd


MAX_TRY = 1000_000
DEBUG = False

random.seed(MAX_TRY)

if DEBUG:
    CAPITAL_LETTERS = 'ABCD'
    ROTOR_TOTAL = 1
    plaintext = "abcd"
    init_rotor_position = 'A'
else:
    CAPITAL_LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    ROTOR_TOTAL = 3
    init_rotor_position = 'AAA'
    plaintext = "The quick brown fox jumps over the lazy dog."
    

def create_rotors(rotor_total):
    # create a dataframe, use index as index, I as the alphabet column name
    index = list(CAPITAL_LETTERS)
    df_rotors = pd.DataFrame(index=index)
    for roter_i in range(rotor_total):
        try_cnt = 0
        alphabet = list(CAPITAL_LETTERS)
        while try_cnt < MAX_TRY:
            try_cnt += 1
            if try_cnt > MAX_TRY:
                print('Exceed max try')
                break
            else:
                alphabet = list(CAPITAL_LETTERS)
                random.shuffle(alphabet)
                differences = [i!=j for i, j in zip(index, alphabet)]
                if all(differences):
                    break # all True means: no same letter at the same positio, e.g. A->A, B->B
                else:
                    continue
        # create a dictionary to map the original letter to the new letter
        # dict_alphabet = dict(zip(index, alphabet))
        # print(alphabet)
        # convert roter_i into Roman number, e.g. I, II, III, IV, V
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][roter_i]
        df_rotors[roter_roman] = alphabet
        # df_rotors[roter_roman + "_id"] = index
        # df_rotors[roter_roman + "_value"] = alphabet
    return df_rotors  


def create_reflector():
    # Reflector is a special rotor, it does not rotate, it mapps the input to the output, e.g. A->Z, B->Y, C->X, etc.
    # But No self mapping, e.g. A->A, B->B, C->C, etc.
    # create 13 pairs from the 26 letters
    alphabet = list(CAPITAL_LETTERS)
    # random.seed(MAX_TRY) # set the seed for reproducibility
    random.shuffle(alphabet) 
    pairs = [alphabet[i:i+2] for i in range(0, len(alphabet), 2)]
    # convert paris into a dictionary
    pairs = dict(pairs)
    # reverse the dictionary to make sure the value of the key is the key of the value
    pairs_reverse = {}
    for k, v in pairs.items():
        pairs_reverse[v] = k
    # merge the two dictionaries
    pairs.update(pairs_reverse)
    # sort the dictionary by the key
    pairs = dict(sorted(pairs.items()))
    # save pairs to a dataframe
    df_reflector = pd.DataFrame(pairs, index=['reflector'])    
    df_reflector = df_reflector.T
    # df_reflector = df_reflector.reset_index(drop=False)
    return df_reflector


def create_rotors_reverse(df_rotors):
    df_temp = df_rotors.copy() 
    df_rotors_reverse = df_rotors.copy() 
    df_temp.reset_index(inplace=True)

    for i in range(ROTOR_TOTAL):
        # select column index and rotor i column
        rotor_name = ['I', 'II', 'III', 'IV', 'V'][i]
        df_temp_pair = df_temp[['index', rotor_name]]

        # sort df_temp_pair by the roter_name column
        df_temp_pair = df_temp_pair.sort_values(by=rotor_name)
        # rename columns 
        df_temp_pair.columns = [rotor_name, 'index']   

        # delete the rotor_name column from df_rotors_reverse
        df_rotors_reverse = df_rotors_reverse.drop(columns=rotor_name)

        # add the column rotor_name to the df_rotors_reverse
        df_rotors_reverse[rotor_name+'_r'] = df_temp_pair[rotor_name].values
        # reverse the order of columns
    df_rotors_reverse = df_rotors_reverse.loc[:, ::-1] 
    return df_rotors_reverse
 
##  Assembleing the rotors, reflector and reverse rotors
# stack the df_rotor and df_rotor_reverse horizontailly
def assemble_engima(df_rotors, df_reflector):
    df_rotors_reverse = create_rotors_reverse(df_rotors)
    df_rotor_reflector_encoder = pd.concat([df_rotors, df_reflector, df_rotors_reverse], axis=1)
    # chent the index name to step  
    df_rotor_reflector_encoder.rename(columns={'index':'name'}, inplace=True)
    return df_rotor_reflector_encoder 


def test_mapping_letter(df_enigma, verbose=False):
    for i, letter in enumerate(CAPITAL_LETTERS):
        # forwad mapping
        letter_mapped = mapping_letter(letter, df_enigma)
        if verbose:
            print(f'forward:\t{letter} -> {letter_mapped}')
        letter2 = letter_mapped
        # backward mapping
        letter_mapped2 = mapping_letter(letter2, df_enigma)
        if verbose:
            print(f'backward:\t{letter2} -> {letter_mapped2}')
        assert letter == letter_mapped2, f'{letter} is not equal to {letter_mapped2}'



# assume the 0 row is the initial position of the rotors
# read the 0 raw all the {I, II, II}_id columns from the df_roters_stepped
def calcualte_rotor_steps(init_rotor_position):
    roters_current_setting_id = list("A"*len(init_rotor_position))
    roters_target_setting_id = list(init_rotor_position)
    roters_steps = [ord(target) - ord(current) for target, current in zip(roters_target_setting_id, roters_current_setting_id)]
    print(f'{roters_current_setting_id} -> {roters_target_setting_id} = {roters_steps}')
    return roters_steps

def test_calcualte_rotor_steps(init_rotor_position = 'AAA'):
    # default df_rotors status is "AAA"
    # from A to Z means 25 steps of change
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [0, 0, 0], f'roters_steps={roters_steps}'

    init_rotor_position = 'BAA' # move one step   
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [1, 0, 0], f'roters_steps={roters_steps}'

    init_rotor_position = 'HIJ' #  
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [7, 8, 9], f'roters_steps={roters_steps}'

    init_rotor_position = 'YBF' #  
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [24, 1, 5], f'roters_steps={roters_steps}'

    init_rotor_position = 'ZZZ'
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [25, 25, 25], f'roters_steps={roters_steps}'

def set_enigma(df_enigma, init_rotor_position = 'AAA'):
    """ set the rotors to the initial position 
    """
    df_enigma_set = df_enigma.copy()
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    
    for i in range(ROTOR_TOTAL):
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][i]
        roter_roman_reverse = roter_roman + '_r'
        for step in range(roters_steps[i]):
            # get the top row, columns [col_rotor_name, col_roter_value]
            rotor_id = df_enigma_set.iloc[0][roter_roman]
            rotor_value = df_enigma_set.iloc[0][roter_roman_reverse]
            col_rotor = [roter_roman, roter_roman_reverse]
            # print(roter_roman, roter_roman, roter_roman_reverse)
            # shift A->B, B->C, C->D, ... Z->A    
            df_enigma_set[col_rotor] = df_enigma_set[col_rotor].shift(-1)
            # backfill the NaN with the 1st row of column [col_rotor_name, roter_roman_reverse]
            df_enigma_set[col_rotor] = df_enigma_set[col_rotor].fillna({roter_roman:rotor_id, roter_roman_reverse:rotor_value})
    return df_enigma_set


def format_input(plaintext):
    input = plaintext
    input = input.upper().replace(' ', '').replace('.', '').replace(',', '').replace('!', '').replace('?', '') 
    # convert into list
    return list(input)

 
def mapping_letter(letter, df_enigma):
    df_enigma = df_enigma.T # transpose the dataframe
    steps_total = df_enigma.shape[0]
    letter_in = letter
    for step in range(steps_total):
        letter_out = df_enigma[letter_in].values[step]
        # print(f'{step= : }    {letter_in} -> {letter_out}')
        letter_in = letter_out # update: out -> in
    return letter_out

def encode_freeze(plaintext, df_engima):
    input = format_input(plaintext)
    df_engima_start = df_engima.copy()
    output = []
    for letter in input:
        output_letter = mapping_letter(letter, df_engima_start)
        output.append(output_letter)
    output = ''.join(output)
    return output

def enigma_stepper(df_engima, roters_steps):    
    df_enigma_next = df_engima.copy()
    for i in range(ROTOR_TOTAL):
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][i]
        roter_roman_reverse = roter_roman + '_r'    
        for step in range(roters_steps[i]): # repeat the steps
            # get the top row, columns [col_rotor_name, col_roter_value]
            rotor_id = df_enigma_next.iloc[0][roter_roman]
            rotor_value = df_enigma_next.iloc[0][roter_roman_reverse]
            col_rotor = [roter_roman, roter_roman_reverse]
            df_enigma_next[col_rotor] = df_enigma_next[col_rotor].shift(-1)
            refill_dict = {roter_roman:rotor_id, roter_roman_reverse:rotor_value} 
            df_enigma_next[col_rotor] = df_enigma_next[col_rotor].fillna(refill_dict)
    return df_enigma_next

def encode_live(plaintext, df_engima):
    input = format_input(plaintext)
    output = []
    base, column = df_engima.shape
    counter = 0
    rotor_cnt = [0, 0, 0]
    for letter in input:
        counter = counter + 1
        rotor_cnt[0] = counter % base
        if rotor_cnt[0] == base - 1:
            rotor_cnt[1] = (rotor_cnt[1] +  1) % base
        if rotor_cnt[1] == base - 1:
            rotor_cnt[2] = (rotor_cnt[2] +  1) % base
        if rotor_cnt[2] == base - 1:
            rotor_cnt[2] = 0
        # rotating the rotors 1 step
        df_engima_next = enigma_stepper(df_engima, rotor_cnt)

        # encode the next letter
        output_letter = mapping_letter(letter, df_engima_next)
        output.append(output_letter) 

        # show output
        # print(f"{letter =} {rotor_cnt = }")
        # display(df_engima_next)
    # list to string
    output = ''.join(output)
    return output, counter, rotor_cnt

# display(df_rotors)
# display(df_reflector)
 
df_rotors = create_rotors(rotor_total=ROTOR_TOTAL)
df_reflector = create_reflector() 
df_enigm_assemble = assemble_engima(df_rotors, df_reflector)
df_enigma = set_enigma(df_enigm_assemble, init_rotor_position)

display(df_enigma)


In [ ]:
test_calcualte_rotor_steps(init_rotor_position=init_rotor_position)
test_mapping_letter(df_enigma) 
 

In [ ]:
# encrypt the plaintext into ciphertext
for i in range(2):
    ciphertext = encode_freeze(plaintext, df_enigma)
    message = encode_freeze(ciphertext, df_enigma)
    # 
    print(f'{"="*25} Round {i} {"="*25}')
    print(f'{plaintext  =}')
    print(f'{ciphertext =}')
    print(f'{message    =}')

for i in range(2):
    ciphertext, ciphertext_cnt, ciphertext_rotor_cnt = encode_live(plaintext, df_enigma)
    message, message_cnt, message_rotor_cnt = encode_live(ciphertext, df_enigma)    # 
    print(f'{"="*25} Round {i} {"="*25}')
    # input = format_input(plaintext)
    # input = ''.join(input)
    # print(f'{input      =} {len(input)}')
    print(f'{plaintext  =}')
    print(f'{ciphertext =} {ciphertext_cnt}:{ciphertext_rotor_cnt}' )
    print(f'{message    =} {message_cnt}:{message_rotor_cnt}')



## Save the Engima

In [ ]:
# Save the csv file
engima_file = 'engima.csv'
df_enigma.to_csv(engima_file, index=True, index_label='index')
# df_rotor_reflector_encoder

##  Simple PlainText Test

In [ ]:
from pathlib import Path

# load the engima_file file if it exists
if Path(engima_file).exists():
    df_enigma = pd.read_csv(engima_file, index_col=0)
    # display(df_enigma)
else:
    print(f'{engima_file} does not exist')
    # create a new engima file


In [ ]:
# encrypt the plaintext into ciphertext
ciphertext = encode_freeze(plaintext, df_enigma)

print(f'{plaintext  =}')
print(f'{ciphertext =}')

# decrypt the ciphertext into message by just put the ciphertext into the encode function again.
message = encode_freeze(ciphertext, df_enigma)
print(f'{message    =}')